<a href="https://colab.research.google.com/github/ah20776/CE807---Assignment/blob/main/Final/CE807_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis
!pip install gensim
!pip install spacy

In [2]:
# Load news data set
# remove meta data headers footers and quotes from news dataset

from pprint import pprint
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True,
                            random_state=32,
                            remove=('headers', 'footers', 'qutes'))

#
dataset_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=32, remove=('headers', 'footers', 'qutes'))
dataset_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=32, remove=('headers', 'footers', 'qutes'))
# Check the names of the categories
pprint(dataset.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [4]:

'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
import nltk
nltk.download('wordnet')
import pandas as pd
stemmer = SnowballStemmer("english")
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
from spacy.lang.en.examples import sentences

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [7]:
processed_docs = []

for doc in dataset_train.data:
    processed_docs.append(preprocess(doc))

'''
Preview 'processed_docs'
'''
pprint(processed_docs[:2])

[['real',
  'question',
  'opinion',
  'motorola',
  'processor',
  'run',
  'compar',
  'intel',
  'processor',
  'run',
  'window',
  'recal',
  'convers',
  'run',
  'window',
  'benchmark',
  'speed',
  'know',
  'true',
  'love',
  'hear',
  'technic',
  'data',
  'david'],
 ['current',
  'street',
  'price',
  'follow',
  'relev',
  'tax',
  'simm',
  'simm',
  'refund',
  'possibl',
  'export',
  'recommend',
  'reliabl',
  'supplier']]


In [23]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
bigram = gensim.models.Phrases(processed_docs, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[processed_docs], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc)) 
   if word not in stop_words] for doc in texts]
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
   [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
      doc = nlp(" ".join(sent))
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out
data_words_nostops = remove_stopwords(processed_docs)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[
   'NOUN', 'ADJ', 'VERB', 'ADV'
])
print(data_lemmatized[:4]) #it will print the lemmatized data.

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['real', 'question', 'opinion', 'window', 'recal', 'conver', 'run', 'window', 'benchmark', 'speed', 'know', 'true', 'love', 'hear', 'technic'], ['current', 'street', 'price', 'follow', 'refund', 'reliabl', 'supplier'], ['help', 'inform', 'card', 'reader', 'recent', 'buy', 'local', 'surplus', 'dealer', 'rear', 'follow', 'inform', 'card', 'reader', 'connector', 'power', 'connector'], ['write', 'sick', 'call', 'legisl', 'unseal', 'involv', 'atroc', 'includ', 'presid', 'attorney_general', 'governor', 'suspend', 'pend', 'serious', 'doubt']]


In [32]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:4]) #it will print the corpus we created above.
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]] 
#it will print the words with their frequencies.
lda_model = gensim.models.ldamodel.LdaModel(
   corpus=corpus, id2word=id2word, num_topics=10, random_state=100, 
   update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True
)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2)], [(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)], [(15, 1), (21, 1), (22, 2), (23, 2), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1)], [(33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)]]


In [33]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.076*"right" + 0.030*"discuss" + 0.030*"case" + 0.026*"probabl" + '
  '0.025*"public" + 0.023*"group" + 0.022*"stop" + 0.022*"author" + '
  '0.020*"clear" + 0.019*"hold"'),
 (1,
  '0.040*"obvious" + 0.037*"opinion" + 0.035*"grind" + 0.031*"agre" + '
  '0.028*"plan" + 0.026*"rememb" + 0.023*"land" + 0.023*"decid" + 0.022*"mile" '
  '+ 0.020*"metal"'),
 (2,
  '0.083*"look" + 0.065*"help" + 0.062*"good" + 0.039*"send" + 0.039*"post" + '
  '0.029*"order" + 0.027*"month" + 0.026*"sell" + 0.020*"guess" + '
  '0.019*"model"'),
 (3,
  '0.041*"problem" + 0.035*"drive" + 0.033*"work" + 0.031*"high" + '
  '0.025*"card" + 0.024*"support" + 0.022*"need" + 0.020*"control" + '
  '0.017*"driver" + 0.016*"type"'),
 (4,
  '0.033*"exist" + 0.032*"argument" + 0.029*"claim" + 0.028*"true" + '
  '0.023*"accept" + 0.020*"believ" + 0.019*"wrong" + 0.016*"religion" + '
  '0.016*"truth" + 0.015*"reason"'),
 (5,
  '0.043*"list" + 0.039*"mail" + 0.034*"line" + 0.031*"inform" + 0.025*"space" '
  '+ 0.024*

In [34]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.934359046077182


In [35]:
coherence_model_lda = CoherenceModel(
   model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3884752068244352


In [36]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, 
    update_every=1, chunksize=100, passes=10, alpha=a, eta=b, per_word_topics=True)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [40]:
import numpy as np
import tqdm
from google.colab import files

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 0
max_topics = 30
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    files.download('lda_tuning_results.csv')
    pbar.close()




  0%|          | 0/300 [00:00<?, ?it/s]


  0%|          | 0/300 [00:06<?, ?it/s]
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


  0%|          | 1/300 [00:53<4:24:19, 53.04s/it]

  1%|          | 2/300 [01:49<4:29:15, 54.21s/it]

  1%|          | 3/300 [02:50<4:37:42, 56.10s/it]

  1%|▏         | 4/300 [03:44<4:33:07, 55.36s/it]

  2%|▏         | 5/300 [04:36<4:28:10, 54.54s/it]

  2%|▏         | 6/300 [05:34<4:31:18, 55.37s/it]

  2%|▏         | 7/300 [06:34<4:37:45, 56.88s/it]

  3%|▎         | 8/300 [07:37<4:46:28, 58.86s/it]

  3%|▎         | 9/300 [08:42<4:53:40, 60.55s/it]

  3%|▎         | 10/300 [09:39<4:48:10, 59.62s/it]

  4%|▎         | 11/300 [10:33<4:38:45, 57.87s/it]

  4%|▍         | 12/300 [11:30<4:35:36, 57.42s/it]

  4%|▍         | 13/300 [12:29<4:37:25, 58.00s/it]

  5%|▍         | 14/300 [13:27<4:36:54, 58.09s/it]

  5%|▌         | 15/300 [14:21<4:30:15,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

100%|██████████| 300/300 [8:04:34<00:00, 96.91s/it] 


In [4]:
print('Number of unique tokens: %d' % len(id2word)) #dictionary
print('Number of documents: %d' % len(corpus))

NameError: ignored

In [ ]:
from pyLDAvis import gensim_models as gensimvis
pyLDAvis.enable_notebook()

vis_data = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis_data)